In [1]:
import numpy 
with open('full_brain_volume_annotated.npy', 'rb') as file:
    full_brain_volume_annotated = numpy.load(file)

In [2]:
from cloudvolume import CloudVolume

path = 'file:///home/sherry/Projects/pipeline_utility/Litao/test'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(path, mip=0, info=info, compress=False, progress=False)
vol.commit_info()
vol[:, :, :] = full_brain_volume_annotated[:, :, :]

In [ ]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

cloudpath = 'file:///home/sherry/Projects/pipeline_utility/Litao/test'
tq = LocalTaskQueue(parallel=4)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, progress=True, compress=None)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)

Downloading: 100%|██████████| 19/19 [00:41<00:00,  2.20s/it]

Downloading: 100%|██████████| 38/38 [00:50<00:00,  1.33s/it]

Downloading: 100%|██████████| 38/38 [00:52<00:00,  1.37s/it]

Downloading:  38%|███▊      | 29/76 [00:56<02:06,  2.70s/it]


Mesh: 100%|██████████| 4/4 [00:03<00:00,  1.27it/s]

Mesh: 100%|██████████| 9/9 [00:03<00:00,  2.47it/s]

Mesh: 100%|██████████| 8/8 [00:03<00:00,  2.19it/s]

Downloading:  29%|██▉       | 11/38 [00:00<00:01, 19.66it/s]

In [4]:
help(tc.create_meshing_tasks)

Help on function create_meshing_tasks in module igneous.task_creation:

create_meshing_tasks(layer_path, mip, shape=(448, 448, 448), simplification=True, max_simplification_error=40, mesh_dir=None, cdn_cache=False, dust_threshold=None, object_ids=None, progress=False, fill_missing=False, encoding='precomputed', spatial_index=True, sharded=False, compress='gzip')

